In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('./Dataset/IMDB Dataset.csv')

data['review'] = data['review'].str.lower()


In [5]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [6]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result


In [7]:
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
data_without_stopwords.head()

,review,sentiment,review without stopwords,clean_review
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...


In [9]:

reviews = data_without_stopwords['clean_review']
reviews

0        one reviewers mentioned watching just 1 oz epi...
1        wonderful little production  the filming techn...
2        thought wonderful way spend time hot summer we...
3        basically family little boy  jake  thinks zomb...
4        petter mattei s  love time money  visually stu...
                               ...                        
49995    thought movie right good job  wasn t creative ...
49996    bad plot  bad dialogue  bad acting  idiotic di...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    no one expects star trek movies high art  fans...
Name: clean_review, Length: 50000, dtype: object

In [10]:
reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])



In [11]:
sentiment = data_without_stopwords['sentiment']

In [12]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

In [13]:
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)

In [14]:
len(Y_train)

40000

In [15]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [16]:
words_to_index = tokenizer.word_index

In [17]:
len(words_to_index)

95419

In [18]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map
    

In [19]:
word_to_vec_map = read_glove_vector('./Dataset/glove.6B.100d.txt')

In [20]:
maxLen = 150


In [21]:

vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)






In [22]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [23]:
model = imdb_rating((maxLen,))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 100)          9541900   
                                                                 
 lstm (LSTM)                 (None, 150, 128)          117248    
                                                                 
 dropout (Dropout)           (None, 150, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 150, 128)          131584    
                                                                 
 dropout_1 (Dropout)         (None, 150, 128)          0         
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584

In [24]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [25]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(40000, 150)

In [26]:
import tensorflow
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(X_train_indices, Y_train, batch_size=64, epochs=5)

Epoch 1/5
625/625 [==============================] - 25s 26ms/step - loss: 0.5653 - accuracy: 0.6987
Epoch 2/5
625/625 [==============================] - 17s 27ms/step - loss: 0.4403 - accuracy: 0.7978
Epoch 3/5
625/625 [==============================] - 16s 26ms/step - loss: 0.4117 - accuracy: 0.8138
Epoch 4/5
625/625 [==============================] - 17s 28ms/step - loss: 0.3907 - accuracy: 0.8261
Epoch 5/5
625/625 [==============================] - 16s 26ms/step - loss: 0.3658 - accuracy: 0.8381


In [28]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [29]:
model.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 5s 12ms/step - loss: 0.3598 - accuracy: 0.8376


[0.35979047417640686, 0.8375999927520752]

In [31]:
preds = model.predict(X_test_indices)

In [32]:
n = np.random.randint(0,9999)

X_test[n]


' the f b i   starring reed hadley  all star guest cast  film begins accidental  convenient   kidnapping  leads one thing  another   doesn t really indicate main story   big house  u s a   prison break story  story improbable  say least  like tv show   violent   for times  but   cast trip  picture this  ralph meeker sent prison  cell mates following criminals  broderick crawford  lon chaney jr   charles bronson  reading  muscle  magazine    william talman  reading  detective  magazine    honest  know that  early scene reveals happens  missing  boy  answering ending  voiceover   don t want hanging  don t miss opening scenes  iceman  boy  peter votrian well runaway asthmatic       big house  u s a   1955  howard w  koch   broderick crawford  ralph meeker  reed hadley'

In [33]:
if preds[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')


precicted sentiment : negative
correct sentiment : negative


In [34]:
preds[n]

array([0.06034359], dtype=float32)

In [35]:
Y_test[n]

0

In [36]:
reviews_list_idx = tokenizer.texts_to_sequences(reviews_list)

In [37]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data

  

In [38]:
data = add_score_predictions(data, reviews_list_idx)

In [39]:
data

,review,sentiment,review without stopwords,clean_review,sentiment score,predicted sentiment
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...,0.672862,positive
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...,0.957684,positive
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...,0.967898,positive
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...,0.011554,negative
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...,0.971591,positive
...,...,...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,thought movie right good job. wasn't creative ...,thought movie right good job wasn t creative ...,0.857205,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di...",bad plot bad dialogue bad acting idiotic di...,0.013096,negative
49997,i am a catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...,catholic taught parochial elementary schools n...,0.310850,negative
49998,i'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,going disagree previous comment side maltin on...,0.414135,negative


# Conceptnet embeddings below:

In [40]:
word_to_vec_map = read_glove_vector('./Dataset/numberbatch-en-19.08.txt')

In [41]:
maxLen = 150


In [42]:

vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)






In [43]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [44]:
model = imdb_rating((maxLen,))
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 300)          28625700  
                                                                 
 lstm_3 (LSTM)               (None, 150, 128)          219648    
                                                                 
 dropout_2 (Dropout)         (None, 150, 128)          0         
                                                                 
 lstm_4 (LSTM)               (None, 150, 128)          131584    
                                                                 
 dropout_3 (Dropout)         (None, 150, 128)          0         
                                                                 
 lstm_5 (LSTM)               (None, 128)               1315

In [45]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [46]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(40000, 150)

In [47]:
import tensorflow
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
model.fit(X_train_indices, Y_train, batch_size=64, epochs=5)

Epoch 1/5
625/625 [==============================] - 20s 27ms/step - loss: 0.4507 - accuracy: 0.7725
Epoch 2/5
625/625 [==============================] - 17s 27ms/step - loss: 0.3462 - accuracy: 0.8489
Epoch 3/5
625/625 [==============================] - 17s 28ms/step - loss: 0.3358 - accuracy: 0.8537
Epoch 4/5
625/625 [==============================] - 17s 27ms/step - loss: 0.3292 - accuracy: 0.8584
Epoch 5/5
625/625 [==============================] - 17s 27ms/step - loss: 0.3239 - accuracy: 0.8594


In [49]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [50]:
model.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 5s 12ms/step - loss: 0.3185 - accuracy: 0.8620


[0.31847143173217773, 0.8619999885559082]

In [52]:
preds = model.predict(X_test_indices)

In [53]:
n = np.random.randint(0,9999)

X_test[n]


'excellent film  momento  nolan s big budget film  much better   recommend people go see momento like that  see film  film shot black white bit annoyed first film understand black white best way film seen  extremely gripping reasonably easy understand even though way made extremely clever  elements storyline think bit daft film definitely worthy second viewing  conculde film clever plot  clever twists turns  good acting bearing mind budget film say pretty amazing '

In [54]:
if preds[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')


predicted sentiment : positive
correct sentiment : positive


In [55]:
preds[n]

array([0.9958768], dtype=float32)

In [56]:
Y_test[n]

1

In [57]:
reviews_list_idx = tokenizer.texts_to_sequences(reviews_list)

In [58]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score_conceptnet'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['sentiment score_conceptnet'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment_conceptnet'] = 0

  data['predicted sentiment_conceptnet'] = pred_sentiment

  return data

  

In [59]:
data = add_score_predictions(data, reviews_list_idx)

In [60]:
data

,review,sentiment,review without stopwords,clean_review,sentiment score,predicted sentiment,sentiment score_conceptnet,predicted sentiment_conceptnet
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...,0.672862,positive,0.876590,positive
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...,0.957684,positive,0.990779,positive
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...,0.967898,positive,0.984460,positive
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...,0.011554,negative,0.064410,negative
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...,0.971591,positive,0.976048,positive
...,...,...,...,...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,thought movie right good job. wasn't creative ...,thought movie right good job wasn t creative ...,0.857205,positive,0.984591,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di...",bad plot bad dialogue bad acting idiotic di...,0.013096,negative,0.007809,negative
49997,i am a catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...,catholic taught parochial elementary schools n...,0.310850,negative,0.523027,positive
49998,i'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,going disagree previous comment side maltin on...,0.414135,negative,0.194901,negative
